# Setup stuff: Connectivity

In [1]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


In [2]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [3]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [4]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


<_Graph my_graph, 2oQasu531XJPg7jwnV2hssaKkMR1eBhPZKcXcTcjzzTu, 2>


#  Actual analytics ..

In [5]:

from katana.remote import analytics


l_cntr = 20                                                #  Used to generate a unique column name
   #
print("--")


--


In [6]:

#  Getting the airport node id
#

#  l_airport = "MKE"
#  l_airport = "ORD"
#  l_airport = "DEN"
l_airport = "SJC"


l_query  = """
   MATCH (n: Airport) 
   WHERE n.airport_code = '{0}'
   RETURN ID(n) AS my_id
   """.format(l_airport)

l_airport_id = my_graph.query(l_query)["my_id"][0]

print("Airport id:  %s   %d" % (l_airport, l_airport_id))
print("")


          0/? [?op/s]

Airport id:  SJC   1



In [7]:

#  Running analytics.sssp() ..

#  From above
#
l_source_nodeid       = str(l_airport_id)


#  Below:  this currently fails on DISTANCE|NUM_HOPS.  Works with None.
#

#  Edge property
#
#  l_weight_property     = "DISTANCE"                 #  Edge property
l_weight_property     = "NUM_HOPS"
#   l_weight_property     = None   

l_max_iterations      = 10000 


#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)


analytics.sssp(
   my_graph,
      #
   source               = l_source_nodeid,
   weight_property      = l_weight_property,
   max_iterations       = l_max_iterations,
   result_property_name = l_output_propname,
   )

print("--")


#  Error,
#
#     ---------------------------------------------------------------------------
#     CancelledError                            Traceback (most recent call last)
#     Cell In [47], line 27
#          23    #
#          24 l_output_propname     = "sssp_dist_" + str(l_cntr).zfill(4)
#     ---> 27 analytics.sssp(
#          28    my_graph,
#          29       #
#          30    source               = l_source_nodeid,
#          31    weight_property      = l_weight_property,
#          32    max_iterations       = l_max_iterations,
#          33    result_property_name = l_output_propname,
#          34    )
#          36 print("--")
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:168, in async_to_sync.<locals>.wrapper(timeout, *args, **kwargs)
#         #     165 if convert_objects:
#         166     registry = AsyncToSyncClassRegistry.get()
#         167     return registry.async_to_sync(
#     --> 168         wait_for(
#         169             async_func(
#         170                 *(registry.sync_to_async(a) for a in args),
#         171                 **{k: registry.sync_to_async(v) for k, v in kwargs.items()},
#         172             ),
#         173             timeout=timeout,
#         174         )
#         175     )
#         176 return wait_for(async_func(*args, **kwargs), timeout=timeout)
#     
#     File /opt/conda/lib/python3.8/site-packages/katana_enterprise/async_to_sync.py:147, in wait_for(coro, timeout)
#         145 try:
#         146     future = asyncio.run_coroutine_threadsafe(timeout_coro, loop=AsyncRunnerThread.get().loop)
#     --> 147     return future.result()
#         148 except KeyboardInterrupt:
#         149     inner_future.cancel()
#     
#     File /opt/conda/lib/python3.8/concurrent/futures/_base.py:442, in Future.result(self, timeout)
#         439 self._condition.wait(timeout)
#         441 if self._state in [CANCELLED, CANCELLED_AND_NOTIFIED]:
#     --> 442     raise CancelledError()
#         443 elif self._state == FINISHED:
#         444     return self.__get_result()
#     
#     CancelledError: 
    

          0/? [?op/s]

CancelledError: 

In [ ]:

#  analytics.sssp() adds a new property to the node
#

l_result = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  l_result2 = my_graph.query(
#     """
#     MATCH (n) - [r] -> (m)
#     RETURN r
#     """ )
#  print(tabulate(l_result2, headers='keys', tablefmt='psql'))
